# Proceso de ETL con los datos disponibles

### Extracción (Extract)

In [ ]:
import pandas as pd

# Cargar los archivos CSV en DataFrames
df_australian_user_reviews = pd.read_csv('Datos-Csv/australian_users_reviews.csv')
df_australian_users_items = pd.read_csv('Datos-Csv/australian_users_items.csv')
df_output_steam_games = pd.read_csv('Datos-Csv/output_steam_games.csv')


##### Validacion 

In [ ]:
df_australian_user_reviews.head()

In [ ]:
df_australian_users_items.head()

In [ ]:
df_output_steam_games.head()

### Transformación (Transform)

##### Funciones de normalizacion

In [ ]:
import pandas as pd

#normalizacion en general
def normalize_dataframe(df, null_tolerance, column_name=None, min_chars=None):
    # Eliminar duplicados
    df.drop_duplicates(inplace=True)

    # Eliminar filas con más de 'null_tolerance' valores nulos
    df.dropna(thresh=null_tolerance, inplace=True)

    # Eliminar filas que contienen listas vacías
    for col in df.columns:
        df = df[df[col].apply(lambda x: x != [] if isinstance(x, list) else True)]

    # Si se proporciona un nombre de columna y un mínimo de caracteres,
    # eliminar las filas donde el contenido de esa columna tiene menos caracteres que 'min_chars'
    if column_name and min_chars:
        df = df[df[column_name].str.len() >= min_chars]

    # Si se proporciona un nombre de columna,
    # eliminar las filas donde el contenido de esa columna es una lista vacía
    if column_name:
        df = df[df[column_name].apply(lambda x: x != [] if isinstance(x, list) else True)]

    return df


In [ ]:
import pandas as pd

#normalizacion de fechas
def normalize_date_column(df, date_column):
    # Intentamos convertir la columna de fechas al formato deseado
    try:
        df[date_column] = pd.to_datetime(df[date_column], format='%Y-%m-%d')
    except ValueError:
        # Si la conversión falla, eliminamos las filas con fechas en un formato incorrecto
        df = df[pd.to_datetime(df[date_column], errors='coerce').notna()]
    return df


##### output_steam_games_normalizado

In [ ]:
df_output_steam_games_normalizado=normalize_dataframe(df_output_steam_games,5)
df_output_steam_games_normalizado=normalize_date_column(df_output_steam_games,'release_date')

In [ ]:
df_output_steam_games_normalizado.to_csv("Datos-Normalizados/output_steam_games_normalizado.csv")

##### australian_user_items_normalizado

In [ ]:
df_australian_users_items_normalizado=normalize_dataframe(df_australian_users_items,0,'items',4)

In [ ]:
df_australian_users_items_normalizado

In [ ]:
df_australian_users_items_normalizado.to_csv('Datos-Normalizados/australian_users_items_normalizado.csv')

##### australian_user_reviews_normalizado

Análisis de las emociones de las reseñas

In [ ]:
import pandas as pd
from textblob import TextBlob

# Asegurarse de que 'reviews' sea una lista de diccionarios
df_australian_user_reviews['reviews'] = df_australian_user_reviews['reviews'].apply(lambda x: eval(x) if pd.notna(x) else [])

# Extraer el contenido de la columna 'review'
df_australian_user_reviews['review_content'] = df_australian_user_reviews['reviews'].apply(lambda reviews: [review['review'] for review in reviews])

# Función para realizar el análisis de sentimiento y devolver el puntaje
def get_sentiment_score(review):
    analysis = TextBlob(str(review))
    if analysis.sentiment.polarity > 0:
        return 2  # Positivo
    elif analysis.sentiment.polarity == 0:
        return 1  # Neutral
    else:
        return 0  # Negativo

# Aplicar el análisis de sentimiento y reemplazar el texto en 'review' con el puntaje
df_australian_user_reviews['reviews'] = df_australian_user_reviews['reviews'].apply(lambda reviews: [{'sentiment_analysis': get_sentiment_score(review['review']), **review} for review in reviews])

# Imprimir las primeras filas del DataFrame resultante
print(df_australian_user_reviews[['reviews']])


In [29]:
df_australian_user_reviews[['reviews']]

0        [{'sentiment_analysis': 2, 'funny': '', 'poste...
1        [{'sentiment_analysis': 2, 'funny': '', 'poste...
2        [{'sentiment_analysis': 2, 'funny': '', 'poste...
3        [{'sentiment_analysis': 2, 'funny': '', 'poste...
4        [{'sentiment_analysis': 1, 'funny': '3 people ...
                               ...                        
25794    [{'sentiment_analysis': 2, 'funny': '', 'poste...
25795    [{'sentiment_analysis': 1, 'funny': '', 'poste...
25796    [{'sentiment_analysis': 1, 'funny': '1 person ...
25797    [{'sentiment_analysis': 2, 'funny': '', 'poste...
25798    [{'sentiment_analysis': 2, 'funny': '1 person ...
Name: reviews, Length: 25799, dtype: object

In [27]:
df_australian_user_reviews.to_csv('Datos-Normalizados/australian_user_reviews_normalizado.csv')

### Carga (Load)